# TensorFlow Lite Model for Deployment

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
# Load the model
model = keras.models.load_model('xception_v5_34_0.897.h5')

In [3]:
# Download image for prediction
!wget http://bit.ly/mlbookcamp-pants -O pants.jpg

--2022-11-15 13:02:18--  http://bit.ly/mlbookcamp-pants
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg [following]
--2022-11-15 13:02:18--  https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23048 (23K) [image/jpeg]
Saving to: 'pants.jpg'

     0K .......... .......... ..                              100%  998K=0.02s

2022-11-15 13:02:21 (998 KB/s) - 'pants.jpg'

In [4]:
# Import libraries to load image and preprocessing
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [5]:
# Load the image with expect image size
img = load_img('pants.jpg', target_size=(299, 299))

# Convert image to numpy array and add batch dimension
x = np.array(img)
X = np.array([x])

# Preprocess image
X = preprocess_input(X)

In [6]:
# Make model prediction on preprocessed image
preds = model.predict(X)[0]
preds

1/1 [==============================] - 4s 4s/step


array([-4.4526634, -7.497402 , -4.0071654, -4.0869155, 11.95938  ,
       -3.2584846, -4.643536 ,  1.930926 , -5.294927 , -4.093612 ],
      dtype=float32)

In [7]:
# List to classes the model was trained to predict
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

# Check the prediction scores
dict(zip(classes, preds))

{'dress': -4.4526634,
 'hat': -7.497402,
 'longsleeve': -4.0071654,
 'outwear': -4.0869155,
 'pants': 11.95938,
 'shirt': -3.2584846,
 'shoes': -4.643536,
 'shorts': 1.930926,
 'skirt': -5.294927,
 't-shirt': -4.093612}

## Convert Keras Model to TF-Lite

In [8]:
# Initial tf-lite converter for keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Make conversion
tflite_model = converter.convert()

# Save the model in tflite format
with open('clothing-model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\awon\AppData\Local\Temp\tmpamzg4ke4\assets


INFO:tensorflow:Assets written to: C:\Users\awon\AppData\Local\Temp\tmpamzg4ke4\assets


It changes the keras model into TensorFlow save model and then save it into tf-lite format. That's why we got the warnings above.

In [9]:
# Check the saved model and its size
!ls -lh

total 163M
-rw-r--r-- 1 awon 197121 81M Nov 15 13:03 clothing-model.tflite
-rw-r--r-- 1 awon 197121 23K Nov 15 13:02 pants.jpg
-rw-r--r-- 1 awon 197121 12K Nov 15 13:02 tensorflow-lite-model.ipynb
-rw-r--r-- 1 awon 197121 83M Nov 12 23:07 xception_v5_34_0.897.h5


Let's load the tf-lite model and use it.

In [10]:
# Import tensorflow lite module
import tensorflow.lite as tflite

In [11]:
# Instantiate model using Interpreter class
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
# Load the weights from the model to memory
interpreter.allocate_tensors()

# Get the input index from interpreter
input_index = interpreter.get_input_details()[0]['index']
# Get the output index from interpreter
output_index = interpreter.get_output_details()[0]['index']

In [12]:
# Set the value of the input tensor using input_index to the image X
interpreter.set_tensor(input_index, X)
# Invoke the interpreter
interpreter.invoke()
# Get the value of the output tensor using output_index to make prediction
preds = interpreter.get_tensor(output_index)[0]
preds

array([-4.4526663, -7.4974055, -4.0071616, -4.0869083, 11.95938  ,
       -3.2584872, -4.643537 ,  1.930927 , -5.2949286, -4.09361  ],
      dtype=float32)

In [13]:
# Check the prediction scores
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds))

{'dress': -4.4526663,
 'hat': -7.4974055,
 'longsleeve': -4.0071616,
 'outwear': -4.0869083,
 'pants': 11.95938,
 'shirt': -3.2584872,
 'shoes': -4.643537,
 'shorts': 1.930927,
 'skirt': -5.2949286,
 't-shirt': -4.09361}

We get the same predictions as before but this time with the tensorflow lite model

## Removing TF Dependencies

We still have few problems with the work we have done when converting the model to tensorflow lite, we still need the `load_img` and `preprocess_input` functions from tensorflow to make the image ready for model predictions and we want to avoid using tensorflow for deployement.

Therefore, we want to load image using `PIL` and to create custom function for image preprocessing.

**Reference**:

- [Keras utils to load image using PIL](https://github.com/keras-team/keras-preprocessing/blob/master/keras_preprocessing/image/utils.py)
- [Image preprocessing](https://github.com/keras-team/keras/blob/master/keras/applications/imagenet_utils.py)

In [14]:
# Import Image class from PIL
from PIL import Image
from PIL.Image import Resampling

In [15]:
# Read image using Image class
with Image.open('pants.jpg') as img:
    # Expected image size and resampling filter
    img = img.resize((299, 299), resample=Resampling.NEAREST)

In [16]:
# Function for image preprocessing
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [17]:
# Preprocess the image
x = np.array(img, dtype='float32') # must be float type
X = np.array([x])
X = preprocess_input(X)
X

array([[[[-0.11372548, -0.15294117, -0.19999999],
         [-0.11372548, -0.15294117, -0.19999999],
         [-0.10588235, -0.14509803, -0.19215685],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.04313725, -0.04313725, -0.10588235],
         [-0.11372548, -0.11372548, -0.17647058]],

        [[-0.09019607, -0.12941176, -0.17647058],
         [-0.09019607, -0.12941176, -0.17647058],
         [-0.08235294, -0.12156862, -0.16862744],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.04313725, -0.04313725, -0.10588235],
         [-0.10588235, -0.10588235, -0.16862744]],

        [[-0.09803921, -0.1372549 , -0.18431371],
         [-0.09803921, -0.1372549 , -0.18431371],
         [-0.09019607, -0.12941176, -0.17647058],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.03529412, -0.03529412, -0.09803921],
         [-0.09019607, -0.09019607, -0.15294117]],

        ...,

        [[-0.67058825, -0.7019608 , -0

In [18]:
# Make prediction on image using tf-lite model
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)[0]
preds

array([-4.4526663, -7.4974055, -4.0071616, -4.0869083, 11.95938  ,
       -3.2584872, -4.643537 ,  1.930927 , -5.2949286, -4.09361  ],
      dtype=float32)

In [19]:
# Check predictions
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds))

{'dress': -4.4526663,
 'hat': -7.4974055,
 'longsleeve': -4.0071616,
 'outwear': -4.0869083,
 'pants': 11.95938,
 'shirt': -3.2584872,
 'shoes': -4.643537,
 'shorts': 1.930927,
 'skirt': -5.2949286,
 't-shirt': -4.09361}

## Simpler Way of Loading and Preprocessing Image

There is even simpler way of loading and preprocessing image using `keras-image-helper` from the project [here](https://github.com/alexeygrigorev/keras-image-helper).

In order to use keras-image-helper library we need to install it:

In [23]:
!pip install keras-image-helper

Our model needs tf-lite from tensorflow package, however, we can use tf-lite without depending on tensorflow and this is what we are looking for. For this, we need to install `tflite-runtime` from [GitHub Coral Python](https://github.com/google-coral/py-repo) page:

In [24]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/
     ------------------------------------ 867.1/867.1 kB 869.9 kB/s eta 0:00:00


Now, instead of importing `import tensorflow.lite as tflite` which comes from tensorflow, we only want to import `import tflite_runtime.interpreter as tflite` from the tflite_runtime.

In [1]:
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [2]:
# Initalize interpreter
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
# Allocate memory
interpreter.allocate_tensors()

# Get input index
input_index = interpreter.get_input_details()[0]['index']
# Get output index
output_index = interpreter.get_output_details()[0]['index']

In [3]:
# Create preprocessor for the model we used for training (Xception)
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [4]:
# Image url for making prediction
url = 'http://bit.ly/mlbookcamp-pants'

# Preprocess the image
X = preprocessor.from_url(url) # there is also a method called 'from_path'
X

array([[[[-0.11372548, -0.15294117, -0.19999999],
         [-0.11372548, -0.15294117, -0.19999999],
         [-0.10588235, -0.14509803, -0.19215685],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.04313725, -0.04313725, -0.10588235],
         [-0.11372548, -0.11372548, -0.17647058]],

        [[-0.09019607, -0.12941176, -0.17647058],
         [-0.09019607, -0.12941176, -0.17647058],
         [-0.08235294, -0.12156862, -0.16862744],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.04313725, -0.04313725, -0.10588235],
         [-0.10588235, -0.10588235, -0.16862744]],

        [[-0.09803921, -0.1372549 , -0.18431371],
         [-0.09803921, -0.1372549 , -0.18431371],
         [-0.09019607, -0.12941176, -0.17647058],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.03529412, -0.03529412, -0.09803921],
         [-0.09019607, -0.09019607, -0.15294117]],

        ...,

        [[-0.67058825, -0.7019608 , -0

In [5]:
# Make prediction on image like befor
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)[0]
preds

array([-4.452664 , -7.497402 , -4.0071664, -4.0869126, 11.959379 ,
       -3.258485 , -4.643536 ,  1.9309283, -5.2949286, -4.0936103],
      dtype=float32)

In [6]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds))

{'dress': -4.452664,
 'hat': -7.497402,
 'longsleeve': -4.0071664,
 'outwear': -4.0869126,
 'pants': 11.959379,
 'shirt': -3.258485,
 'shoes': -4.643536,
 'shorts': 1.9309283,
 'skirt': -5.2949286,
 't-shirt': -4.0936103}